In [1]:
import os, json, math
import pandas as pd
import numpy as np
#from neo4j import GraphDatabase
from py2neo import Graph, NodeMatcher
from pprint import pprint
import datetime
import random, string

base='/Users/m088378/Documents/Chen_Node2Vec/OneDrive_1_9-24-2020/'
mutPatho = pd.read_csv(base+'pathogenic.tsv', sep='\t', low_memory=False)
mutVus = pd.read_csv(base+'vus.tsv', sep='\t', low_memory=False)

# https://maayanlab.cloud/Harmonizome/documentation
# https://github.com/neo4j-contrib/neo4j-graph-algorithms

In [2]:
# set up authenticated connection
#graph = GraphDatabase.driver("bolt://localhost:7687/PO2_Neo", auth=("neo4j", "ngs4"))
graph = Graph("bolt://localhost:7687/PO2_Neo", auth=("neo4j", "ngs4"))

In [3]:
p2 = pd.melt(mutPatho, id_vars=['newid'], value_vars=mutPatho.columns[1:],var_name='Genes')
p2 = p2[-pd.isnull(p2.value)]
p2

,newid,Genes,value
486,F00505,ABL1,Y253C
742,F00763,ABL1,K609del
925,F00946,ABL1,G807_P810del
981,F01002,ABL1,K609del
1083,F01104,ABL1,P441L
...,...,...,...
1170135,F02479,ZNF703,amplification
1170174,F02518,ZNF703,amplification
1170276,F02620,ZNF703,amplification
1170365,F02710,ZNF703,amplification


In [4]:
#for row in p2.head(100).itertuples():
for row in p2.itertuples():
    #pprint(row)
    s1 = "MERGE (u:Patient { StudyId: '%s'})"%(row.newid)
    s2 = "MERGE (g:Gene { Symbol: '%s'})"%(row.Genes)
    s3 = "MERGE (m:Mutation { Term: '%s'})"%(row.value)
    r1 = "MERGE (m)-[:WITHIN]->(g)"
    r2 = "MERGE (m)-[:PATHOGENIC]->(u)"
    subStmt = "{} {} {} {} {}".format(s1,s2,s3,r1,r2)
    graph.run(subStmt)
## Create 8363 Nodes, 20,583 Edges across 3 lables, 2 relationships

In [5]:
#stm = '''
#CREATE CONSTRAINT ON (g:Gene)
#ASSERT g.Symbol IS UNIQUE
#'''
#graph.run(stm)

In [6]:
# Creating indexes on fields that would be used frequently for filtering.
#graph.run("CREATE INDEX ON :Patient(StudyId)")
#graph.run("CREATE INDEX ON :Gene(Symbol)")
#graph.run("CREATE INDEX ON :Mutation(Term)")
# ClientError: [Schema.EquivalentSchemaRuleAlreadyExists] An equivalent index already 
#exists, 'Index( id=1, name='index_b2dc2555', type='GENERAL BTREE', schema=(:Patient 
#{StudyId}), indexProvider='native-btree-1.0' )'.

In [7]:
p3 = pd.melt(mutVus, id_vars=['newid'], value_vars=mutVus.columns[1:],var_name='Genes')
p3 = p3[-pd.isnull(p3.value)]
p3

,newid,Genes,value
17,F00030,ABL1,Y257N
53,F00068,ABL1,G864S
132,F00147,ABL1,W687*
162,F00178,ABL1,E197K
174,F00190,ABL1,K1029*
...,...,...,...
1435811,F02786,ZNF703,A401_H402insPTHLGGSSCSTCSA
1435816,F02791,ZNF703,G344S
1435817,F02792,ZNF703,A401_H402insPTHLGGSSCSTCSA
1435839,F02815,ZNF703,"A500fs*43,G439V"


In [8]:
for row in p3.itertuples():
    #pprint(row)
    s1 = "MERGE (u:Patient { StudyId: '%s'})"%(row.newid)
    s2 = "MERGE (g:Gene { Symbol: '%s'})"%(row.Genes)
    s3 = "MERGE (m:Mutation { Term: '%s'})"%(row.value)
    r1 = "MERGE (m)-[:WITHIN]->(g)"
    r2 = "MERGE (m)-[:VUS]->(u)"
    subStmt = "{} {} {} {} {}".format(s1,s2,s3,r1,r2)
    graph.run(subStmt)
## Create 25,101 Nodes, 64,844 Edges across 3 lables, 3 relationships

In [9]:
fndTable = pd.read_csv(base+'FoundationSum_Aug24_2020.txt', sep='\t', low_memory=False)
#fndTable.head(5)["ProvidedDiagnosis"]
fndTable.iloc[:, [0,26]]

#list(fndTable.columns.values)

,newid,ProvidedDiagnosis
0,F01262,Esophagus adenocarcinoma[8/28/16]
1,F02558,Unknown primary adenocarcinoma[4/26/18]
2,F00915,Breast invasive lobular carcinoma (ILC)[5/10/19]
3,F01596,Unknown primary adenocarcinoma[3/2/17]
4,F01420,Lymph node lymphoma follicular lymphoma[11/21/16]
...,...,...
2603,F00855,Stomach adenocarcinoma (NOS)[4/22/19]
2604,F00666,Rectum adenocarcinoma (CRC)[2/7/19]
2605,F00760,Lung adenocarcinoma[3/12/19]
2606,F00698,Pancreas ductal adenocarcinoma[2/14/19]


In [10]:
idx = fndTable.columns.get_loc("MedCur[Date]")
for row in fndTable.fillna('').itertuples():
    s1 = "MERGE (u:Patient { StudyId: '%s'})"%(row.newid)
    if(row[idx]):
        for ele in row[idx].split(','):
            arr = ele.replace(']','').split('[')
            date_time_obj = datetime.datetime.strptime(arr[1], '%m/%d/%y')    
            s2 = "MERGE (d:AdministrationDate { Date: '%s'})"%(date_time_obj.date())
            s3 = "MERGE (m:Medication { Name: '%s', RecordType: 'MedCur'})"%(arr[0])
            r1 = "MERGE (m)-[:ADMINISTER_ON]->(d)"
            r2 = "MERGE (d)-[:ADMINISTER_TO]->(u)"
            subStmt = "{} {} {} {} {}".format(s1,s2,s3,r1,r2)
            graph.run(subStmt)

In [11]:
idx = fndTable.columns.get_loc("MedUDP[Date]")
for row in fndTable.fillna('').itertuples():
    s1 = "MERGE (u:Patient { StudyId: '%s'})"%(row.newid)
    if(row[idx]):
        for ele in row[idx].split(','):
            arr = ele.replace(']','').split('[')
            date_time_obj = datetime.datetime.strptime(arr[1], '%m/%d/%y')    
            s2 = "MERGE (d:AdministrationDate { Date: '%s'})"%(date_time_obj.date())
            s3 = "MERGE (m:Medication { Name: '%s', RecordType: 'MedUDP'})"%(arr[0])
            r1 = "MERGE (m)-[:ADMINISTER_ON]->(d)"
            r2 = "MERGE (d)-[:ADMINISTER_TO]->(u)"
            subStmt = "{} {} {} {} {}".format(s1,s2,s3,r1,r2)
            graph.run(subStmt)
            
idx = fndTable.columns.get_loc("MedHx[Date]")
for row in fndTable.fillna('').itertuples():
    s1 = "MERGE (u:Patient { StudyId: '%s'})"%(row.newid)
    if(row[idx]):
        for ele in row[idx].split(','):
            arr = ele.replace(']','').split('[')
            date_time_obj = datetime.datetime.strptime(arr[1], '%m/%d/%y')    
            s2 = "MERGE (d:AdministrationDate { Date: '%s'})"%(date_time_obj.date())
            s3 = "MERGE (m:Medication { Name: '%s', RecordType: 'MedHx'})"%(arr[0])
            r1 = "MERGE (m)-[:ADMINISTER_ON]->(d)"
            r2 = "MERGE (d)-[:ADMINISTER_TO]->(u)"
            subStmt = "{} {} {} {} {}".format(s1,s2,s3,r1,r2)
            graph.run(subStmt)

In [12]:
for row in fndTable.itertuples():
    s1 = "MERGE (u:Patient { StudyId: '%s'})"%(row.newid)
    if(row.ProvidedDiagnosis):
        for ele in row.ProvidedDiagnosis.split(','):
            arr = ele.replace(']','').split('[')
            date_time_obj = datetime.datetime.strptime(arr[1], '%m/%d/%y')    
            s2 = "MERGE (d:DiagnosisDate { Date: '%s'})"%(date_time_obj.date())
            s3 = "MERGE (m:Diagnosis { Name: '%s', RecordType: 'MayoDiagnosis'})"%(arr[0])
            r1 = "MERGE (m)-[:DIAGNOSED_ON]->(d)"
            r2 = "MERGE (d)-[:DIAGNOSED_TO]->(u)"
            subStmt = "{} {} {} {} {}".format(s1,s2,s3,r1,r2)
            #print(subStmt+"\n")
            graph.run(subStmt)

In [15]:
for row in fndTable.itertuples():
    s1 = "MERGE (u:Patient { StudyId: '%s'})"%(row.newid)
    if(row.AgeAtTest):
        s2 = "MERGE (y:Age { Year: '%s'})"%(row.AgeAtTest)
        r1 = "MERGE (u)-[:AGE_AT_TEST]->(y)"
        subStmt = "{} {} {}".format(s1,s2,r1)
        #print(subStmt+"\n")
        graph.run(subStmt)
        

In [14]:
matcher = NodeMatcher(graph)
for row in fndTable.itertuples():
    thisdict = {
      "BirthDate": row.PATIENT_BIRTH_DATE,
      "Gender": row.PATIENT_GENDER_NAME,
      "Race": row.PATIENT_RACE_NAME,
      "PrimaryState" : row.PATIENT_PRIMARY_STATE_CODE,  
      "Visits": row.visit
    }
    result = matcher.match('Patient', StudyId=row.newid).first()
    if(result):
        result.update(thisdict)
        graph.push(result)
    else:
        thisdict['JoinerID'] = ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16))
        print(row.newid)
        for dualID in row.newid.split(','):
            result = matcher.match('Patient', StudyId=dualID).first()
            if(result):
                result.update(thisdict)
                graph.push(result)

#pprint(result)